In [1]:
from numpy.random import seed
seed(1)
import scipy.spatial as sp
from tensorflow import set_random_seed
set_random_seed(2)
import numpy as np, pandas as pd
from collections import Counter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Add, concatenate , Subtract
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, BatchNormalization,Lambda
from keras.models import Model
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.dummy import DummyClassifier
import json
# np.random.seed(44)
from sklearn.utils import shuffle
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))


/usr/lib64/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/usr/lib64/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib64/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [2]:
#Function to load embeddings
def load_embeddings(path, dimension):
    f = open(path, encoding="utf8").read().splitlines()
    vectors = {}
    for i in f:
        elems = i.split()
        vectors[" ".join(elems[:-dimension])] =  np.array(elems[-dimension:]).astype(float)
    return vectors

In [3]:
#Load Embeddings
embeddings = load_embeddings("glove.6B.300d.txt", 300)

In [4]:
# load embeddings
#We include the concatenation, the difference or Cosine. All the combinations are possible.

def get_vector_representation_of_word_pairs(dataframe, embeddings_voca,concat,diff,Cosine):
    x1 = [embeddings_voca[word] for word in dataframe.w1.values]
    x2 =[embeddings_voca[word] for word in dataframe.w2.values]
    y = dataframe.Category.values
    #Concatenation
    if concat :
        x = np.hstack((x1, x2))
    #Difference
    if diff :
        z=np.subtract(x1, x2)
        if concat :
            x= np.hstack((x,z))
        else :
            x=z
    #Cosine
    if Cosine :
        cosine=np.diag(1 - sp.distance.cdist(x1,x2, 'cosine'))
        cosine=np.reshape(cosine,(-1,1))
        x= np.hstack((x,cosine)) 

    return x, y



In [13]:
#Perpare_data




        
        
#Read the data and the Output is 3 Dataframes : df1 for task1 ,df2 for task2 and df3 for Random Pairs
def three_data(Rumen,Root9,Bless,Cogalex,Weeds):
    def prep_df(df):
        cols =df.columns
        for i in range(len(cols)) :
            for j in range(len(df)):
                df[df.columns[i]].values[j]=df[df.columns[i]].values[j][0:-2]
        return df
    if Weeds:
        Rumen=True
        task1="HYPER"
        task2="Coord"
        link2="data/coordpairs2_wiki100.json"
        link1="data/entpairs2_wiki100.json"
    if Cogalex :
        task1="HYPER"
        task2="SYN"
        link1="CogALexV_train_v1/gold_task2.txt"
        link2="CogALexV_test_v1/gold_task2.txt"
    if Rumen :
        task1="HYPER"
        task2="SYN"
        link="RumenPairs.txt"
    if Root9 :
        link_hyper="MULTITASK-LEARNING/ROOT9/ROOT9_hyper.txt"
        link_coord="MULTITASK-LEARNING/ROOT9/ROOT9_coord.txt"
        link_random="MULTITASK-LEARNING/ROOT9/ROOT9_random.txt"
        task1= "HYPER"
        task2= "COORD"
    if Bless :
        task1= "HYPER"
        task2= "MERO"
        link_coord="MULTITASK-LEARNING/BLESS/BLESS_mero.txt"
        link_hyper="MULTITASK-LEARNING/BLESS/BLESS_hyper.txt"
        link_random="MULTITASK-LEARNING/BLESS/BLESS_random.txt"

    def get_names(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task1
        if cat == 2: return task2
    def get_names_Weeds1(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task1
    def get_names_Weeds2(cat):
        if cat == 0 : return "RANDOM"
        if cat == 1: return task2




    if Rumen :
        dff = pd.read_csv(link)
        dff.rename(columns={"W1":"w1", "W2":"w2","rel":"Category"}, inplace=True)
        dff["Category"] = dff["Category"].apply(get_names)
        df = dff.loc[dff.Category == task2]
        df2 = dff.loc[dff.Category == task1]
        df3 = dff.loc[dff.Category == "RANDOM"]
        #print(len(df),len(df2),len(df3))
    if Root9 or Bless:

        df = pd.read_csv(link_coord,header=None,sep = '\t')
        df.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df=prep_df(df)
        df2 = pd.read_csv(link_hyper,header=None,sep = '\t')
        df2.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df2=prep_df(df2)
        df3 = pd.read_csv(link_random,header=None,sep = '\t')
        df3.rename(index=str,columns={0:"w1", 2:"w2",1:"Category"},inplace=True)
        df3=prep_df(df3)
    if Cogalex:

        dff1 = pd.read_csv(link1,header=None,sep = '\t')
        dff2 = pd.read_csv(link2,header=None,sep = '\t')
        dff1.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff2.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff3=pd.concat([dff1,dff2])
        #print(list(set(dff3.Category.values.tolist())))
        df = dff3.loc[dff3.Category == task2]
        df2 = dff3.loc[dff3.Category == task1]
        df3 = dff3.loc[dff3.Category == "RANDOM"]
    if Weeds :
        json_data=open(link1).read()
        data = json.loads(json_data)
        dff=pd.DataFrame(data)
        dff.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff["Category"] = dff["Category"].apply(get_names_Weeds1)
        df2 = dff.loc[dff.Category == task1]
        #df3 = dff.loc[dff.Category == "RANDOM"]
        #df3=df3[0:len(df2)]
        #print("taille 0,1 pour entpairs",len(df2),len(df3))
        
        json_data2=open(link2).read()
        data2 = json.loads(json_data2)
        dff2=pd.DataFrame(data2)
        dff2.rename(index=str,columns={0:"w1", 1:"w2",2:"Category"},inplace=True)
        dff2["Category"] = dff2["Category"].apply(get_names_Weeds2)
        df = dff2.loc[dff2.Category == task2]
        #df=df[0:len(df2)]
        #print("taille 0,1 pour coord",len(df))
        
        
    return df,df2,df3



In [14]:
#Construct the train and test data based on Lexical Split
def func_train_test(name_data,embeddings,concat,diff,cosine):
    
    Cogalex,Rumen,Root9,Weeds,Bless=False,False,False,False,False
    if str(name_data)=="Cogalex" :
        Cogalex=True
        #print('Preprocessing of Cogalex ')
    if str(name_data)=="Weeds" :
        Weeds=True
        #print('Preprocessing of Weeds ')
        
    if str(name_data)=="Rumen" :
        Rumen=True
        #print('Preprocessing of Rumen ')
    if str(name_data)=="Root9" :
        Root9=True
        #print('Preprocessing of Root9 ')
    if str(name_data)=="Bless" :
        Bless=True
        #print('Preprocessing of Bless')
        
    df,df2,df3=three_data(Rumen,Root9,Bless,Cogalex,Weeds)
 
    words_coord = list(set(df.w1.values.tolist() + df.w2.values.tolist()))
    words_hyper = list(set(df2.w1.values.tolist() + df2.w2.values.tolist()))
    words_random = list(set(df3.w1.values.tolist() + df3.w2.values.tolist()))

    words_ = sorted(list(set(words_coord+words_hyper+words_random)))
    #words_ = sorted(list(set(words_coord+words_hyper+words_random1+words_random2)))
    
    #LExical SPlit
    words_train, words_test =train_test_split(words_, test_size=0.4, random_state=1344)
    #print('numbers_words_task1_not_sorted: '+str(len(list((df.w1.values.tolist() + df.w2.values.tolist())))))
    #print('numbers_words_sorted: '+str(len(words_)) )
    #print('numbers_words_task1_sorted: '+str(len(words_coord)))
    #print('numbers_words_task2_sorted: '+str(len(words_hyper)) )
    #print('numbers_words_random_sorted: '+str(len(words_random)) +'\n')

    
    #print('numbers_pairs_task1_all: '+str(len(df)) )
    #print('numbers_pairs_task2_all: '+str(len(df2)) )
    #print('numbers_pairs_random_all: '+str(len(df3)) +'\n')
  

    df["known_words"] = df.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    df2["known_words"] = df2.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    df3["known_words"] = df3.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    
    
    #print('numbers_pairs_task1_known: '+str(len(df)) )
    #print('numbers_pairs_task2_known: '+str(len(df2)) )
    #print('numbers_pairs_random_known: '+str(len(df3)) +'\n')
    
    
    # Given the words in the train and test parts, mark the pairs as training or testing, when both words of aa pair belong to the train or test vocabulary.
    df["is_train"] = df.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df["is_test"] = df.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)
 


    

    df2["is_train"] = df2.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df2["is_test"] = df2.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)
 
    
    df3["is_train"] = df3.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
    df3["is_test"] = df3.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)

    xtrainCoord, ytrainCoord = get_vector_representation_of_word_pairs(df.loc[df.is_train==True], embeddings,concat,diff,cosine)
    xtestCoord, ytestCoord   = get_vector_representation_of_word_pairs(df.loc[df.is_test==True], embeddings,concat,diff,cosine)
    
    #print('numbers_pairs_task1_train: '+str(len(ytrainCoord) ))
    #print('numbers_pairs_task1_test: '+str(len(ytestCoord)) )
    #print('Percentage_pairs_train/test: '+str(len(ytestCoord)*100/(len(ytestCoord)+len(ytrainCoord)))+ '%'+'\n') 
    
    

    xtrainHyper, ytrainHyper = get_vector_representation_of_word_pairs(df2.loc[df2.is_train==True], embeddings,concat,diff,cosine)
    xtestHyper, ytestHyper   = get_vector_representation_of_word_pairs(df2.loc[df2.is_test==True], embeddings,concat,diff,cosine)
          
    #print('numbers_pairs_task2_train: '+str(len(ytrainHyper) ))
    #print('numbers_pairs_task2_test: '+str(len(ytestHyper)) )
    #print('Percentage_pairs_train/test: '+str(len(ytestHyper)*100/(len(ytestHyper)+len(ytrainHyper)))+ '%'+'\n') 
          
          
          
    xtrainRando, ytrainRando = get_vector_representation_of_word_pairs(df3.loc[df3.is_train==True], embeddings,concat,diff,cosine) 
    xtestRando, ytestRando   = get_vector_representation_of_word_pairs(df3.loc[df3.is_test==True], embeddings,concat,diff,cosine)
    
    #print('numbers_pairs_random_train: '+str(len(ytrainRando) ))
    #print('numbers_pairs_random_test: '+str(len(ytestRando)) )
    #print('Percentage_random_train/test: '+str(len(ytestRando)*100/(len(ytestRando)+len(ytrainRando)))+ '%'+'\n') 
          
    x_train_1, x_train_2 = np.vstack((xtrainCoord, xtrainRando)), np.vstack((xtrainHyper, xtrainRando))
    y_train_1, y_train_2 = [1]*len(xtrainCoord) + [0]*len(xtrainRando), [1]*len(xtrainHyper) + [0]*len(xtrainRando)



    x_test_1, x_test_2 = np.vstack((xtestCoord, xtestRando)), np.vstack((xtestHyper, xtestRando))
    y_test_1, y_test_2 = [1]*len(xtestCoord) + [0]*len(xtestRando), [1]*len(xtestHyper) + [0]*len(xtestRando)
    


    x_train_1, y_train_1 = shuffle(x_train_1, y_train_1, random_state=1234)
    x_train_2, y_train_2 = shuffle(x_train_2, y_train_2, random_state=1234)
    x_test_1, y_test_1 = shuffle(x_test_1, y_test_1, random_state=1234)
    x_test_2, y_test_2 = shuffle(x_test_2, y_test_2, random_state=1234)
    assert len(x_train_1) == len(y_train_1)
    assert len(x_train_2) == len(y_train_2)
    assert len(x_test_1) == len(y_test_1)
    assert len(x_test_2) == len(y_test_2)
    data = {}
    for name, x_train, y_train, x_test, y_test in zip(["Coord-Random","Hyper-Random"], [x_train_1, x_train_2], [y_train_1, y_train_2], [x_test_1, x_test_2], [y_test_1, y_test_2]):   
        # Perform the splits in train, validation, unlabeled
        #x_train, x_unlabeled, y_train, y_unlabeled = train_test_split(x_train, y_train, stratify=y_train, test_size=0.6, random_state=1234,)
        x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, stratify=y_train,  test_size=0.30, random_state=1234,)


        # keep the train/validation/test splits so that hey can be used with multitask learning and the results are comparable between them
        #print('name '+name)
        data[name]={"x_train": x_train, "y_train":y_train,   "x_valid":x_valid, "y_valid":y_valid, "x_test":x_test,  "y_test":y_test } 
    
    sum_task1Vsrandom=len(data["Coord-Random"]["y_train"])+len(data["Coord-Random"]["y_test"])+len(data["Coord-Random"]["y_valid"])
    sum_task2Vsrandom=len(data["Hyper-Random"]["y_train"])+len(data["Hyper-Random"]["y_test"])+len(data["Hyper-Random"]["y_valid"])
    #print('sums_pairs_task1Vsrandom :' +str(sum_task1Vsrandom) )
    #print('sums_pairs_task2Vsrandom :' +str(sum_task2Vsrandom) +'\n')
        
    #print('numbers_pairs_task1Vsrandom_train: '+str(len(data["Coord-Random"]["y_train"]) ), 'Percentage :'+ str(len(data["Coord-Random"]["y_train"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_train: '+str(len(data["Hyper-Random"]["y_train"]) ), 'Percentage :'+ str(len(data["Hyper-Random"]["y_train"])*100/sum_task2Vsrandom) +' %'+'\n')
    
    #print('numbers_pairs_task1Vsrandom_valid: '+str(len(data["Coord-Random"]["y_valid"]) ),'Percentage :'+ str(len(data["Coord-Random"]["y_valid"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_valid: '+str(len(data["Hyper-Random"]["y_valid"]) ) , 'Percentage :'+ str(len(data["Hyper-Random"]["y_valid"])*100/sum_task2Vsrandom) +' %'+'\n')
    
    #print('numbers_pairs_task1Vsrandom_test: '+str(len(data["Coord-Random"]["y_test"]) ),'Percentage :'+ str(len(data["Coord-Random"]["y_test"])*100/sum_task1Vsrandom) +' %')
    #print('numbers_pairs_task2Vsrandom_test: '+str(len(data["Hyper-Random"]["y_test"]) ), 'Percentage :'+ str(len(data["Hyper-Random"]["y_test"])*100/sum_task2Vsrandom) +' %'+'\n')
    return data




In [15]:
#These function gives the input dim. It should be imporoved in the future.
def input_dim_func(concat,diff,cosine):
    if concat and diff== False  and cosine==False   :
        input_dim=600
    if concat and diff== False  and cosine   :
        input_dim=601
    if concat and diff and cosine==False   :
        input_dim=900
    if concat and diff and cosine   :
        input_dim=901
    if concat==False and diff and cosine==False   :
        input_dim=300
    if concat==False and diff  and cosine  :
        input_dim=301
    return input_dim

In [16]:
#Simple NN baseline
def get_my_nn_model():
    input_dim=input_dim_func(concat,diff,cosine)
    #print('NN_baseline','input_dim ',input_dim)
    """Defines the NN baseline.
    Two hidden layers, followed by the output layer. 
    """
    model = Sequential()
    model.add(Dense(50, activation='sigmoid', input_dim=input_dim))
    #model.add(Dropout(0.1))
    #model.add(Dense(15, activation='sigmoid'))

    model.add(Dense(1, activation='sigmoid'))
    #model.add(Dropout(0.1))
    model.compile(optimizer='Adam', loss='binary_crossentropy')
    return model

#Multi_task LEarning V0.0
def get_my_multitask_nn_models_ancien():
    #print('Multi Ancien')
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))
    x = Dense(50, activation='sigmoid')(inputs)
    
   
    coord = Dense(1, activation='sigmoid', name='coord_output')(x)
    #coord=Dropout(0.05)(coord)
    hyper = Dense(1, activation='sigmoid', name='hyper_output')(x)
    #hyper=Dropout(0.5)(hyper)
    
    model_coord = Model(inputs=[inputs], outputs=[coord])
    model_hyper = Model(inputs=[inputs], outputs=[hyper])

    model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
    return model_coord, model_hyper

#Multi task Learning V1.1
def get_my_multitask_nn_models():
 
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))

    #The sared layer
    x = Dense(300, activation='relu')(inputs)

                

    #The layer of task1
    concatenated_hyper = concatenate([inputs,x])
    #The layer of task2
    concatenated_coord = concatenate([inputs,x])

    
    xcoord = Dense(50, activation='sigmoid')(concatenated_coord)
    xhyper= Dense(50, activation='sigmoid')(concatenated_hyper)

    
    #Output for task1
    hyper = Dense(1, activation='sigmoid', name='hyper_output')(xhyper)
    model_hyper = Model(inputs=[inputs], outputs=[hyper])    
                
    #Output for task2
    coord = Dense(1, activation='sigmoid', name='coord_output')(xcoord)
    model_coord = Model(inputs=[inputs], outputs=[coord])
    
    


    model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
    return model_coord, model_hyper


#Multi Task V1.1
def get_my_multitask_nn_models_new():
    #print('Multi New2')
    input_dim=input_dim_func(concat,diff,cosine)
    inputs = Input(shape=(input_dim,))
    #This version can be useful only if we have the concatenation for the 2 words.
    if input_dim<600 :
        model_coord, model_hyper= get_my_multitask_nn_models()
        return model_coord, model_hyper
    else :    
    #xcoord0 = Dense(50, activation='sigmoid',kernel_initializer='zeros',bias_initializer='zeros')(inputs)
    #xcoord = Dense(300, activation='sigmoid')(inputs)
    #X1=Dense(25, activation='sigmoid')(xcoord)
    #inputs1=inputs[0:300]
    #inputs2=inputs[300:600]
        inputs1=Lambda(lambda x: x[:,0:300])(inputs)

        inputs2=Lambda(lambda x: x[:,300:600])(inputs)
        inputs3=Lambda(lambda x: x[:,600:])(inputs)
        x = Dense(300, activation='relu')(inputs)
        x1= Dense(100, activation='relu')(inputs1)
        x2= Dense(100, activation='relu')(inputs2)


        concatenated_hyper = concatenate([inputs,x])       
        concatenated_hyper1 = concatenate([concatenated_hyper,x1])
        concatenated_hyper2 = concatenate([concatenated_hyper1,x2])
        concatenated_hyper3 = concatenate([concatenated_hyper2,inputs3])


        concatenated_coord = concatenate([inputs,x])
        concatenated_coord1 = concatenate([concatenated_coord,x1])
        concatenated_coord2 = concatenate([concatenated_coord1,x2])
        concatenated_coord3 = concatenate([concatenated_coord2,inputs3])
        #concatenated = Add()([xcoord,xhyper])
        #concatenated = Subtract()([xcoord,xhyper])
        xcoord = Dense(50, activation='sigmoid')(concatenated_coord3)
        xhyper= Dense(50, activation='sigmoid')(concatenated_hyper3)
        #xcommun= Dense(50,activation='sigmoid')(concatenated)
        #xafter= Dense(25,activation='sigmoid')(xcommun)


        #hyper0 = Dense(5, activation='sigmoid')(concatenated_hyper)
        hyper = Dense(1, activation='sigmoid', name='hyper_output')(xhyper)
        #hyper=Dropout(0.5)(hyper)


        model_hyper = Model(inputs=[inputs], outputs=[hyper])    


        #coord0 = Dense(5, activation='sigmoid')(concatenated_coord)
        coord = Dense(1, activation='sigmoid', name='coord_output')(xcoord)
        model_coord = Model(inputs=[inputs], outputs=[coord])





        model_coord.compile(optimizer='rmsprop', loss='binary_crossentropy')
        model_hyper.compile(optimizer='rmsprop', loss='binary_crossentropy')
        return model_coord, model_hyper

In [26]:
seed(1)
##############1
#Here num_epochs is the number of itreation. Every iteration see one batch.
num_epochs=3000
#seuil = 50
batch=64

#hyp=True
#Coord=True
import time

#Train Model
def train_model(data,name_model,Multi=False):
    tmps1=time.time()

    if Multi==False :
        model_coord = name_model()
        model_hyper = name_model()
    else : model_coord, model_hyper = name_model()




    acc_scores_coord,mif_scores_coord, acc_scores_hyper,  mif_scores_hyper= [], [], [], []
    precision_score_coord,precision_score_hyper,recall_score_hyper,recall_score_coord=[],[],[],[]

    for epoch in range(num_epochs):
        if epoch%500==0 :
            #print (epoch)
            tmps3=time.time()-tmps1
            #print ("Temps d'execution = %f" %tmps3 + "s")
        a1=len(data["Hyper-Random"]["x_train"])
        a2=len(data["Coord-Random"]["x_train"])

        idx1 = np.random.choice(np.arange(a1), batch, replace=False)
        idx2 = np.random.choice(np.arange(a2), batch, replace=False)
       
        model_hyper.fit(data["Hyper-Random"]["x_train"][idx1], np.array(data["Hyper-Random"]["y_train"])[idx1], epochs=1, validation_data=None, verbose=False, )
        model_coord.fit(data["Coord-Random"]["x_train"][idx2], np.array(data["Coord-Random"]["y_train"])[idx2], epochs=1, validation_data=None, verbose=False, )
        #We save the results after seeing 100 batches.
        if epoch%100==0 :
            preds_coord = (model_coord.predict(data["Coord-Random"]["x_test"], verbose=0)> 0.5).astype(int)
            preds_hyper = (model_hyper.predict(data["Hyper-Random"]["x_test"], verbose=0)> 0.5).astype(int)

            preds_coord_valid = (model_coord.predict(data["Coord-Random"]["x_valid"], verbose=0)> 0.5).astype(int)
            preds_hyper_valid = (model_hyper.predict(data["Hyper-Random"]["x_valid"], verbose=0)> 0.5).astype(int)

            acc_scores_coord.append([accuracy_score(data["Coord-Random"]["y_valid"], preds_coord_valid), accuracy_score(data["Coord-Random"]["y_test"], preds_coord)])
            precision_score_coord.append([precision_score(data["Coord-Random"]["y_valid"], preds_coord_valid), precision_score(data["Coord-Random"]["y_test"], preds_coord)])
            recall_score_coord.append([recall_score(data["Coord-Random"]["y_valid"], preds_coord_valid), recall_score(data["Coord-Random"]["y_test"], preds_coord)])
            acc_scores_hyper.append([accuracy_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), accuracy_score(data["Hyper-Random"]["y_test"], preds_hyper)])
            precision_score_hyper.append([precision_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), precision_score(data["Hyper-Random"]["y_test"], preds_hyper)])
            recall_score_hyper.append([recall_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid), recall_score(data["Hyper-Random"]["y_test"], preds_hyper)])

            mif_scores_coord.append([f1_score(data["Coord-Random"]["y_valid"], preds_coord_valid, average="macro") , f1_score(data["Coord-Random"]["y_test"], preds_coord, average="macro")])
            mif_scores_hyper.append([f1_score(data["Hyper-Random"]["y_valid"], preds_hyper_valid, average="macro") , f1_score(data["Hyper-Random"]["y_test"], preds_hyper, average="macro")])

    df1=pd.DataFrame(acc_scores_coord, columns = ['acc_coord_valid', 'acc_coord_test'])
    df2=pd.DataFrame(mif_scores_coord, columns = ['mif_coord_valid', 'mif_coord_test'])
    df3=pd.DataFrame(acc_scores_hyper, columns = ['acc_hyper_valid', 'acc_hyper_test'])
    df4=pd.DataFrame(mif_scores_hyper, columns = ['mif_hyper_valid', 'mif_hyper_test'])
    df5=pd.DataFrame(precision_score_coord, columns = ['prec_coord_valid', 'prec_coord_test'])
    df6=pd.DataFrame(precision_score_hyper, columns = ['prec_hyper_valid', 'prec_hyper_test'])
    df7=pd.DataFrame(recall_score_coord, columns = ['recall_coord_valid', 'recall_coord_test'])
    df8=pd.DataFrame(recall_score_hyper, columns = ['recall_hyper_valid', 'recall_hyper_test'])
    frames=[df1,df2,df3,df4,df5,df6,df7,df8]
    df_final=pd.concat(frames,axis=1)
    acc_coord=df_final['acc_coord_test'][np.argmax(df_final['acc_coord_valid'])]
    acc_hyper=df_final['acc_hyper_test'][np.argmax(df_final['acc_hyper_valid'])]
    mif_coord=df_final['mif_coord_test'][np.argmax(df_final['mif_coord_valid'])]
    mif_hyper=df_final['mif_hyper_test'][np.argmax(df_final['mif_hyper_valid'])]
    prec_coord=df_final['prec_coord_test'][np.argmax(df_final['prec_coord_valid'])]
    prec_hyper=df_final['prec_hyper_test'][np.argmax(df_final['prec_hyper_valid'])]
    #recall_coord=df_final['recall_coord_test'][np.argmax(df_final['prec_coord_valid'])]
    #recall_hyper=df_final['recall_hyper_test'][np.argmax(df_final['prec_hyper_valid'])]
    recall_coord=df_final['recall_coord_test'][np.argmax(df_final['recall_coord_valid'])]
    recall_hyper=df_final['recall_hyper_test'][np.argmax(df_final['recall_hyper_valid'])]
    tmps2=time.time()-tmps1
    tmps2=tmps2/60
    #new_data=[[acc_coord,'Nan'],[acc_hyper,'Nan'],[mif_coord,'Nan'],[mif_coord,'Nan'],[prec_coord,'Nan'],[prec_hyper,'Nan'],[recall_coord,'Nan'],[recall_hyper,'Nan'],[tmps2,'Nan']]
    new_data=[acc_coord,acc_hyper,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,tmps2]
    df_restit=pd.DataFrame(new_data,index=['acc_coord','acc_hyp','mif_coord','mif_hyper','prec_coord','prec_hyper','recall_coord','recall_hyper','time'])
    tmps2=time.time()-tmps1
    tmps2=tmps2/60
    #print ("Temps d'execution = %f" %tmps2 + "min")

    return df_final ,df_restit.transpose()




In [29]:
def main(name_data,name_model,Multi,index):
    evol=[0,1,2,3,4,5]
    index_bool=['FTF','FTT','TFF','TFT','TTF','TTT']


    index_glob=[]
    for i in range(len(index)):
        for j in range(len(index_bool)):
            index_glob.append(index[i]+' '+index_bool[j])
    cols=['acc_coord','acc_hyp','mif_coord','mif_hyper','prec_coord','prec_hyper','recall_coord','recall_hyper','time']
    result=[]
    global concat
    global diff
    global cosine
    for i in range(len(name_model)):
        for j in range(len(evol)):
            if j==0:
                concat,diff,cosine=False,True,False
            if j==1:
                concat,diff,cosine=False,True,True
            if j==2:
                concat,diff,cosine=True,False,False
            if j==3:
                concat,diff,cosine=True,False,True
            if j==4:
                concat,diff,cosine=True,True,False
            if j==5:
                concat,diff,cosine=True,True,True
            data=func_train_test(name_data,embeddings,concat,diff,cosine)
    
            
            df_final,df_restit=train_model(data,name_model[i],Multi[i])
            result.append(list(df_restit.values)[0])
    
    df=pd.DataFrame(result,columns=cols,index=index_glob)
    return df
        
        


In [30]:
index=['NN baseline','Multi_task','Multi_task New','Multi Task New2']
models=[get_my_nn_model,get_my_multitask_nn_models_ancien,get_my_multitask_nn_models,get_my_multitask_nn_models_new]
list_Multi=[False,True,True,True]

In [31]:
df1=main("Rumen",models,list_Multi,index)
df1

/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib64/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,acc_coord,acc_hyp,mif_coord,mif_hyper,prec_coord,prec_hyper,recall_coord,recall_hyper,time
NN baseline FTF,0.675493,0.702227,0.673152,0.699170,0.646643,0.646134,0.851620,0.967436,0.621199
NN baseline FTT,0.809969,0.770223,0.809929,0.766756,0.817773,0.726675,0.824451,0.758480,0.656170
NN baseline TFF,0.758048,0.770809,0.758035,0.767961,0.749746,0.724582,0.780564,0.773406,0.678226
NN baseline TFT,0.823988,0.794842,0.823986,0.792210,0.812695,0.749035,1.000000,0.797829,0.769794
NN baseline TTF,0.760125,0.781360,0.759571,0.778405,0.748992,0.735526,0.920585,1.000000,0.800671
NN baseline TTT,0.819834,0.791911,0.819834,0.789040,0.814346,0.666667,0.827586,0.778833,0.810040
Multi_task FTF,0.690550,0.712778,0.689462,0.710033,0.674835,0.668848,0.777429,0.712347,0.688562
Multi_task FTT,0.802181,0.759672,0.802053,0.755532,0.813043,0.718876,0.808777,0.994573,0.697170
Multi_task TFF,0.751298,0.790152,0.751268,0.787287,0.741414,0.745785,0.784744,0.791045,0.754480
Multi_task TFT,0.823468,0.794842,0.823468,0.792040,0.810726,0.753623,0.840125,0.997286,0.766816
